Grid Search Analysis
===

Compares the results of the grid search per dataset. And spits out the best one...


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
from matplotlib import cm


import json
import codecs
import os

In [170]:
basepath = os.path.normpath("C:\Users\hatieke\.ukpsummarizer\scores_grid")

In [171]:
dirs = [f for f in os.listdir(basepath) if os.path.isdir(os.path.join(basepath, f))]

In [172]:
dirs

['9623ef172d38f5006cb0633b0fcbbbdcbc233ec98b38fbad9db02eef',
 'c9c6a2fcf6977d1e0e888fbea6d352ec8c998736440b75a6bd064534']

In [173]:
selected = dirs[1]
selected

'c9c6a2fcf6977d1e0e888fbea6d352ec8c998736440b75a6bd064534'

In [174]:
def parse_dir(dir):
    p = os.path.join(basepath, dir)
    result_jsons = []
    result_files = [f for f in os.listdir(p) if f.startswith("result-") and f.endswith(".json")]
    for f in result_files:
        fn = os.path.join(p, f)
        fsize = os.path.getsize(fn)
        if fsize > 0:
            with open(fn) as fp:
                result_jsons.append(json.load(fp))
    return result_jsons

In [155]:
f = os.path.join(basepath, selected, "result-af54bffb220ca4e9f459b174b296b3a755f7f1fb1935f3151bc97cd8.json")

In [156]:
f

'C:\\Users\\hatieke\\.ukpsummarizer\\scores_grid\\9623ef172d38f5006cb0633b0fcbbbdcbc233ec98b38fbad9db02eef\\result-af54bffb220ca4e9f459b174b296b3a755f7f1fb1935f3151bc97cd8.json'

In [130]:
with open(f) as fp:
    o = json.load(fp)

In [131]:
o

{u'config_concept_recommendation': {},
 u'config_feedback_interpretation': {},
 u'config_feedbackstore': {u'N': 3,
  u'language': u'english',
  u'multiplier_accept': 0.25,
  u'multiplier_reject': 0.05,
  u'type': u'SimpleNgramFeedbackGraph'},
 u'config_oracle_type': u'accept',
 u'config_parse_type': u'None',
 u'config_run_id': u'af54bffb220ca4e9f459b174b296b3a755f7f1fb1935f3151bc97cd8',
 u'config_summarizer_type': u'PROPAGATION',
 u'config_wordembeddings': u'glove.6B.300d',
 u'dataset': u'DUC2004',
 u'log_feedbacks': [{u'concept': u'the new',
   u'iteration': 0,
   u'value': u'accept'},
  {u'concept': u'expect to', u'iteration': 0, u'value': u'accept'},
  {u'concept': u'new presid', u'iteration': 0, u'value': u'accept'},
  {u'concept': u'civil war', u'iteration': 0, u'value': u'accept'},
  {u'concept': u'elia hrawi', u'iteration': 0, u'value': u'reject'},
  {u'concept': u'general enjoy', u'iteration': 0, u'value': u'reject'},
  {u'concept': u'in rebuild', u'iteration': 0, u'value': u'r

In [157]:
res = [i for i in o["result_rougescores"] if i["iteration"] <= 11]

In [158]:
[i["ROUGE-2 R score"] for i in res]

[0.03883, 0.03883, 0.08738, 0.08738]

In [159]:
[i["ROUGE-2 R score"] for i in o["result_rougescores"] if i["iteration"] <= 11][-1]

0.08738

In [175]:
def parse_single_result_into_dataframe(obj, iteration=11):
    config = obj[u'config_feedbackstore']
    try:
        res = [i for i in obj["result_rougescores"] if i["iteration"] <= iteration][-1]
    except:
        raise BaseException("unknown iteration %s" % (obj["config_run_id"]))
    
    total_accept = sum([1 for i in obj[u'log_feedbacks'] if i["value"] == 'accept' and i["iteration"] < iteration])
    total_reject = sum([1 for i in obj[u'log_feedbacks'] if i["value"] != 'accept' and i["iteration"] < iteration])
    total_feedback = total_accept + total_reject
    
    num_iterations = res["iteration"]
    r1 =  res[u'ROUGE-1 R score']
    r2 =  res[u'ROUGE-2 R score']
    r4 =  res[u'ROUGE-SU* R score']
   
    classtype = config.get(u'type')
    cut_off_threshold = config.get(u'cut_off_threshold')
    iterations_accept = config.get(u'iterations_accept')
    iterations_reject = config.get(u'iterations_reject')
    propagation_abort_threshold = config.get(u'propagation_abort_threshold')
    mass_accept = config.get(u'mass_accept')
    mass_reject = config.get(u'mass_reject')
    window_size = config.get(u'N')
    factor_reject = config.get(u"multiplier_reject")
    factor_accept = config.get(u"multiplier_accept")
    cutoff = config.get(u"cut_off_threshold")
    runid = obj.get("config_run_id")
    
    return {
        "accept" : total_accept,
        "reject": total_reject,
        "total_feedback": total_feedback,
        "ref_summary": str([item["name"] for item in obj[u'models']]),
        "cfg": config,
        "num_iterations": num_iterations,
        "r1": r1,
        "r2": r2,
        "r4": r4,
        "classtype": classtype,
        "iterations_accept":iterations_accept,
        "iterations_reject": iterations_reject,
        "propagation_abort_threshold": propagation_abort_threshold,
        "mass_accept" : mass_accept,
        "mass_reject" : mass_reject,
        "window_size": window_size,
        "multiplier_reject": factor_reject,
        "multiplier_accept": factor_accept,
        "cutoff_threshold": cutoff,
        "id": runid
    }


In [110]:
#parse_single_result_into_dataframe(first, iteration=10)

In [176]:
items = [parse_single_result_into_dataframe(f, iteration=11) for d in dirs for f in parse_dir(d)]
#items = [parse_single_result_into_dataframe(f, iteration=11) for f in parse_dir(selected)]

In [177]:
len(items)

11995

In [178]:
#items[0]

In [179]:
df = pd.DataFrame(items)

In [180]:
df.to_csv("C:\\Users\\hatieke\\.ukpsummarizer\\tmp\\grid.csv")

In [150]:
df.iloc[0]

accept                                                                        22
cfg                            {u'mass_reject': -1.0, u'iterations_accept': 1...
classtype                          WordEmbeddingRandomWalkDiffusionFeedbackGraph
cutoff_threshold                                                             0.6
id                             001ed642fe5d247404e45acb9bdc92717d0d7872ed6a49...
iterations_accept                                                             16
iterations_reject                                                              8
mass_accept                                                                   -1
mass_reject                                                                   -1
multiplier_accept                                                            NaN
multiplier_reject                                                            NaN
num_iterations                                                                 4
propagation_abort_threshold 

In [ ]:
top_score

In [ ]:
v = top_score["cfg"]
v

In [ ]:
v.apply(lambda x: x.mass_reject)

In [ ]:
df.iloc[64]

In [ ]:
df.groupby("ref_summary").max()

In [184]:
df.groupby(["ref_summary","classtype"])["r2"].describe().to_csv("C:\\Users\\hatieke\\.ukpsummarizer\\tmp\\grid_groups.csv")